# Install stan

In [1]:
# Install cmdstanpy module
!pip install cmdstanpy

# Install arviz module (With version 2.33.1 you will need a special version of arviz)
!pip install git+https://github.com/OriolAbril/arviz.git@ci

# Install cmdstan -- Just for the first time!
from cmdstanpy import install_cmdstan
install_cmdstan(compiler=True)

!pip install --upgrade arviz
!pip install --upgrade cmdstanpy

  Cloning https://github.com/OriolAbril/arviz.git (to revision ci) to /tmp/pip-req-build-3rxh8t62
  Running command git clone --filter=blob:none --quiet https://github.com/OriolAbril/arviz.git /tmp/pip-req-build-3rxh8t62
  Running command git checkout -b ci --track origin/ci
  Switched to a new branch 'ci'
  Branch 'ci' set up to track remote branch 'ci' from 'origin'.
  Resolved https://github.com/OriolAbril/arviz.git to commit 7c20182c25b0b5e9af8242d1ee4d1185ae82b70a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for arviz: filename=arviz-0.17.0.dev0-py3-none-any.whl size=1651459 sha256=e905f5a8fda7eb7e2810fa1886546000817064cc09fbfcba5f2c7ea16d4d1079
  Stored in directory: /tmp/pip-ephem-wheel-cache-z531j95n/wheels/46/1e/6c/46b9c646a79d4a0f01b37df1c2c0abcc4e7129fe66e88e9b0f
Successfully built arviz
  Attempting uninstall: arviz
    Found existing installation: arviz 0.15.1
    Unin

DEBUG:cmdstanpy:cmd: make build -j1
cwd: None


Unpacked download as cmdstan-2.34.1
Building version cmdstan-2.34.1, may take several minutes, depending on your system.


DEBUG:cmdstanpy:cmd: make examples/bernoulli/bernoulli
cwd: None


Installed cmdstan-2.34.1
Test model compilation
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: arviz
    Found existing installation: arviz 0.17.0.dev0
    Uninstalling arviz-0.17.0.dev0:
      Successfully uninstalled arviz-0.17.0.dev0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.2.0
    Uninstalling cmdstanpy-1.2.0:
      Successfully uninstalled cmdstanpy-1.2.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
import os

from cmdstanpy import CmdStanModel
from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

# Create ./stan folder if does not exists
if not os.path.exists("/content/drive/My Drive/Bayesian/stan"):
    os.mkdir("/content/drive/My Drive/Bayesian/stan")

# Data preparation

In [14]:
# Import data
data = pd.read_csv('/content/drive/My Drive/Bayesian/All_values_Clean.csv')
data.head(5)

,Unnamed: 0,Id_sensor,Time,NOx,max_wind10,avg_wind10,max_wind100,avg_wind100,max_humidity,avg_humidity,...,sin12,cos12,sin6,cos6,sin4,cos4,sin3,cos3,lat,lng
0,7145,6204,gennaio 16,5.094537,-0.883564,-1.033000,-0.448047,-0.632854,0.614262,0.775022,...,0.500,0.866,0.866,0.5,1,0,0.866,-0.5,45.519335,9.59201
1,7146,6204,febbraio 16,4.847209,-0.160893,-0.054538,0.083888,0.459680,-0.152361,0.826117,...,0.866,0.500,0.866,-0.5,0,-1,-0.866,-0.5,45.519335,9.59201
2,7147,6204,marzo 16,3.911859,0.257496,-0.447750,0.367014,-0.273935,-0.152361,0.137046,...,1.000,0.000,0.000,-1.0,-1,0,0.000,1.0,45.519335,9.59201
3,7148,6204,aprile 16,3.763297,-0.984991,-0.200262,-0.928504,-0.113883,-1.685606,-0.319251,...,0.866,-0.500,-0.866,-0.5,0,1,0.866,-0.5,45.519335,9.59201
4,7149,6204,maggio 16,3.808000,-0.718744,-0.524298,-0.413729,-0.419238,-0.152361,-0.147897,...,0.500,-0.866,-0.866,0.5,1,0,-0.866,-0.5,45.519335,9.59201


In [5]:
unique_values_count = data['Id_sensor'].nunique()

print(f'The number of unique values in the Id_sensor column is: {unique_values_count}')

The number of unique values in the Id_sensor column is: 75


In [15]:
selected_columns = data[['Id_sensor','Time', 'NOx']]
print(selected_columns)

      Id_sensor          Time       NOx
0          6204    gennaio 16  5.094537
1          6204   febbraio 16  4.847209
2          6204      marzo 16  3.911859
3          6204     aprile 16  3.763297
4          6204     maggio 16  3.808000
...         ...           ...       ...
7045      30162     giugno 23  2.918380
7046      30162     luglio 23  2.802844
7047      30162     agosto 23  2.899084
7048      30162  settembre 23  2.765598
7049      30162    ottobre 23  3.381531

[7050 rows x 3 columns]


# First model
y --> N(mu(t), sigma) \\
sigma --> invgamma(3,2) \\

mu(t)=e if t<s \\
mu(t)=l if t>s \\

e --> N(0,1) \\
l--> N(0,1) \\
s--> U(1..T) \\

In [7]:
firstmodel = """
functions {

    // Computes vector of lp
    vector compute_logp(array[] real obs, real early_rate, real late_rate, real sigma){

        // Deduce size of data and uniform contribution
        int T = size(obs);
        real log_unif = -log(T);

        // Compute contributions vector and return
        vector[T] lp = rep_vector(log_unif, T);
        for (s in 1:T) {
            for (t in 1:T) {
                lp[s] += normal_lpdf(obs[t] | t < s ? early_rate : late_rate, sigma);
            }
        }
        return lp;

    }

    // Computes the vector of probabilites for the posterior distribution of s
    vector compute_probs(vector lp) {
        return softmax(lp);
    }

    // Computes the marginal likelihood lpdf
    real marginal_likelihood_logpdf(vector lp){
        return log_sum_exp(lp);
    }

}

data {
    int<lower=1> T; //dimension of the time series
    array[T] real y; //array of y (observations of the number of disaster with mor ethan 10 death)
}

transformed data {
    real log_unif = -log(T);
}

parameters {
    real e;
    real l;
    real <lower=0.0001> sigma;
}

model {
    // Likelihood
    target += marginal_likelihood_logpdf(compute_logp(y, e, l, sigma));

    // Prior
    e ~ normal(0,1);
    l ~ normal(0,1);
    sigma ~ inv_gamma(3,2);

}

generated quantities {
    int<lower=1, upper=T> s = categorical_rng(compute_probs(compute_logp(y,e,l, sigma)));
}
"""

# Write stan model to file
stan_file = "/content/drive/My Drive/Bayesian/stan/firstmodelt.stan"
with open(stan_file, "w") as f:
    print(firstmodel, file=f)

# Compile stan model
firstmodel = CmdStanModel(stan_file=stan_file)

14:52:55 - cmdstanpy - INFO - compiling stan file /tmp/tmpohv_17yk/tmpn31l3a0s.stan to exe file /content/drive/My Drive/Bayesian/stan/firstmodelt
INFO:cmdstanpy:compiling stan file /tmp/tmpohv_17yk/tmpn31l3a0s.stan to exe file /content/drive/My Drive/Bayesian/stan/firstmodelt
DEBUG:cmdstanpy:cmd: make STANCFLAGS+=--filename-in-msg=firstmodelt.stan /tmp/tmpohv_17yk/tmpn31l3a0s
cwd: /root/.cmdstan/cmdstan-2.34.1
DEBUG:cmdstanpy:Console output:

--- Translating Stan model to C++ code ---
bin/stanc --filename-in-msg=firstmodelt.stan --o=/tmp/tmpohv_17yk/tmpn31l3a0s.hpp /tmp/tmpohv_17yk/tmpn31l3a0s.stan

--- Compiling C++ code ---
g++ -std=c++1y -pthread -D_REENTRANT -Wno-sign-compare -Wno-ignored-attributes      -I stan/lib/stan_math/lib/tbb_2020.3/include    -O3 -I src -I stan/src -I stan/lib/rapidjson_1.1.0/ -I lib/CLI11-1.9.1/ -I stan/lib/stan_math/ -I stan/lib/stan_math/lib/eigen_3.4.0 -I stan/lib/stan_math/lib/boost_1.81.0 -I stan/lib/stan_math/lib/sundials_6.1.1/include -I stan/lib/s

In [8]:
import random
random.seed(9)

df=selected_columns
# Ottieni gli ID sensori unici
unique_ids = df['Id_sensor'].unique()

# Seleziona 15 campioni casuali
random_samples = random.sample(list(unique_ids), 20)

# Applied on the entire time series

In [ ]:
def find_maxima(df):
    maxima_info = []

    # Iterate over each unique Id_sensor value
    for id_sensor in random_samples:
        # Select subset for current Id_sensor
        subset_df = df[df['Id_sensor'] == id_sensor]

        # Perform analysis similar to your code
        cpc_data = {
            "T": selected_columns.shape[0],
            "y": selected_columns.NOx,
            "r_e": 1.0,
            "r_l": 1.0
        }


        # Algorithm parameters
        algo_params = {
            "n_chains": 4,
            "n_burnin": 1000,
            "n_iter": 5000
        }

        # Sample
        cpc_fit = firstmodel.sample(data=cpc_data, chains=algo_params["n_chains"], parallel_chains=algo_params["n_chains"],
                                          iter_warmup=algo_params["n_burnin"], iter_sampling=algo_params["n_iter"])
        # Convert to arviz data type
        chains = az.InferenceData(posterior=cpc_fit.draws_xr())
        s_chain = np.hstack(chains.posterior.s).astype(int)
        x, y = np.unique(s_chain, return_counts=True)
        print(y)
        print(id_sensor)
        print("Primo massimo: {0}".format(subset_df['Time'].iloc[x[y == y.max()]].to_numpy().squeeze()))
        sorted_indices = np.argsort(-y)
        # Seleziona il secondo massimo
        second_max_index = sorted_indices[1]
        print("Secondo massimo: {0}".format(subset_df['Time'].iloc[x[second_max_index]]))

        maxima_info.append((id_sensor, subset_df['Time'].iloc[x[y == y.max()]], subset_df['Time'].iloc[x[second_max_index]]))

    return maxima_info

maxima_firstmodel = find_maxima(selected_columns)

# Create dataframes for maxima information
#df_maxima_1 = pd.DataFrame(maxima_area_1, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])
df_maxima_firstmodel = pd.DataFrame(maxima_firstmodel, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])

DEBUG:cmdstanpy:cmd: /content/drive/My Drive/Bayesian/stan/firstmodelt info
cwd: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmpq4vvaw37/7dqmyct7.json
08:32:55 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/drive/My Drive/Bayesian/stan/firstmodelt', 'id=1', 'random', 'seed=69773', 'data', 'file=/tmp/tmpq4vvaw37/7dqmyct7.json', 'output', 'file=/tmp/tmpq4vvaw37/firstmodeltqoeoj13k/firstmodelt-20240208083255_1.csv', 'method=sample', 'num_samples=5000', 'num_warmup=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/drive/My Drive/Bayesian/stan/firstmodelt', 'id=2', 'random', 'seed=69773', 'data', 'file=/tmp/tmpq4vvaw37/7dqmyct7.json', 'output', 'file=/tmp/tmpq4vvaw37/firstmodeltqoeoj13k/firstmodelt-20240208083255_2.csv', 'method=sample', 'num_samples=5000', 'num_warmup=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:idx 3
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-f12283b22950>", line 44, in <cell line: 44>
    maxima_firstmodel = find_maxima(selected_columns)
  File "<ipython-input-9-f12283b22950>", line 26, in find_maxima
    cpc_fit = firstmodel.sample(data=cpc_data, chains=algo_params["n_chains"], parallel_chains=algo_params["n_chains"],
  File "/usr/local/lib/python3.10/dist-packages/cmdstanpy/model.py", line 1123, in sample
    progress_hook = self._wrap_sampler_progress_hook(
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 649, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.10/threading.py", line 1116, i

TypeError: object of type 'NoneType' has no len()

# **applied on the first half of the time series**

In [9]:
random_samples2 = random.sample(list(unique_ids), 15)

In [16]:
# Find the indexes corresponding to 'gennaio 21'
index_gennaio_21 = selected_columns[selected_columns['Time'] == 'gennaio 21'].index[0]

# Create two DataFrames based on indexes
df_subset_1 = selected_columns.loc[:index_gennaio_21]
df_subset_2 = selected_columns.loc[index_gennaio_21:]

# Print or use the subsets as needed
print("Subset 1:")
print(df_subset_1.head())

print("\nSubset 2:")
print(df_subset_2.head())

Subset 1:
   Id_sensor         Time       NOx
0       6204   gennaio 16  5.094537
1       6204  febbraio 16  4.847209
2       6204     marzo 16  3.911859
3       6204    aprile 16  3.763297
4       6204    maggio 16  3.808000

Subset 2:
    Id_sensor         Time       NOx
60       6204   gennaio 21  4.666615
61       6204  febbraio 21  4.452101
62       6204     marzo 21  3.892506
63       6204    aprile 21  3.374293
64       6204    maggio 21  3.151481


In [17]:
def find_maxima(df):
    maxima_info = []

    # Iterate over each unique Id_sensor value
    for id_sensor in random_samples2:
        # Select subset for current Id_sensor
        subset_df = df[df['Id_sensor'] == id_sensor]

        # Perform analysis similar to your code
        cpc_data = {
            "T": selected_columns.shape[0],
            "y": selected_columns.NOx,
            "r_e": 1.0,
            "r_l": 1.0
        }


        # Algorithm parameters
        algo_params = {
            "n_chains": 4,
            "n_burnin": 1000,
            "n_iter": 5000
        }

        # Sample
        cpc_fit = firstmodel.sample(data=cpc_data, chains=algo_params["n_chains"], parallel_chains=algo_params["n_chains"],
                                          iter_warmup=algo_params["n_burnin"], iter_sampling=algo_params["n_iter"])
        # Convert to arviz data type
        chains = az.InferenceData(posterior=cpc_fit.draws_xr())
        s_chain = np.hstack(chains.posterior.s).astype(int)
        x, y = np.unique(s_chain, return_counts=True)
        print(y)
        print(id_sensor)
        print("Primo massimo: {0}".format(subset_df['Time'].iloc[x[y == y.max()]].to_numpy().squeeze()))
        sorted_indices = np.argsort(-y)
        # Seleziona il secondo massimo
        second_max_index = sorted_indices[1]
        print("Secondo massimo: {0}".format(subset_df['Time'].iloc[x[second_max_index]]))

        maxima_info.append((id_sensor, subset_df['Time'].iloc[x[y == y.max()]], subset_df['Time'].iloc[x[second_max_index]]))

    return maxima_info

maxima_firstmodel_split1 = find_maxima(df_subset_1)

# Create dataframes for maxima information
#df_maxima_1 = pd.DataFrame(maxima_area_1, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])
maxima_firstmodel_split1 = pd.DataFrame(maxima_firstmodel_split1, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])

DEBUG:cmdstanpy:cmd: /content/drive/My Drive/Bayesian/stan/firstmodelt info
cwd: None
DEBUG:cmdstanpy:input tempfile: /tmp/tmphecrebea/y9x7o0kp.json
14:57:25 - cmdstanpy - INFO - CmdStan start processing
INFO:cmdstanpy:CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/drive/My Drive/Bayesian/stan/firstmodelt', 'id=1', 'random', 'seed=9785', 'data', 'file=/tmp/tmphecrebea/y9x7o0kp.json', 'output', 'file=/tmp/tmphecrebea/firstmodelt0op5jn7e/firstmodelt-20240208145725_1.csv', 'method=sample', 'num_samples=5000', 'num_warmup=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 1
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/drive/My Drive/Bayesian/stan/firstmodelt', 'id=2', 'random', 'seed=9785', 'data', 'file=/tmp/tmphecrebea/y9x7o0kp.json', 'output', 'file=/tmp/tmphecrebea/firstmodelt0op5jn7e/firstmodelt-20240208145725_2.csv', 'method=sample', 'num_samples=5000', 'num_warmup=1000', 'algorithm=hmc', 'adapt', 'engaged=1']
DEBUG:cmdstanpy:idx 2
DEBUG:cmdstanpy:idx 3
DEBUG:cmdstanpy:running CmdStan, num_threads: 1
DEBUG:cmdstanpy:CmdStan args: ['/content/drive/My Drive/Bayesian/stan/firstmod

KeyboardInterrupt: 

In [13]:
df_subset_1

,Id_sensor,Time,NOx
0,6204,gennaio 16,5.094537
1,6204,febbraio 16,4.847209
2,6204,marzo 16,3.911859
3,6204,aprile 16,3.763297
4,6204,maggio 16,3.808000
...,...,...,...
56,6204,settembre 20,3.497653
57,6204,ottobre 20,4.192372
58,6204,novembre 20,4.648401
59,6204,dicembre 20,4.806568


# **applied on the second half of the time series**

In [ ]:
def find_maxima(df):
    maxima_info = []

    # Iterate over each unique Id_sensor value
    for id_sensor in random_samples2:
        # Select subset for current Id_sensor
        subset_df = df[df['Id_sensor'] == id_sensor]

        # Perform analysis similar to your code
        cpc_data = {
            "T": selected_columns.shape[0],
            "y": selected_columns.NOx,
            "r_e": 1.0,
            "r_l": 1.0
        }


        # Algorithm parameters
        algo_params = {
            "n_chains": 4,
            "n_burnin": 1000,
            "n_iter": 5000
        }

        # Sample
        cpc_fit = firstmodel.sample(data=cpc_data, chains=algo_params["n_chains"], parallel_chains=algo_params["n_chains"],
                                          iter_warmup=algo_params["n_burnin"], iter_sampling=algo_params["n_iter"])
        # Convert to arviz data type
        chains = az.InferenceData(posterior=cpc_fit.draws_xr())
        s_chain = np.hstack(chains.posterior.s).astype(int)
        x, y = np.unique(s_chain, return_counts=True)
        print(y)
        print(id_sensor)
        print("Primo massimo: {0}".format(subset_df['Time'].iloc[x[y == y.max()]].to_numpy().squeeze()))
        sorted_indices = np.argsort(-y)
        # Seleziona il secondo massimo
        second_max_index = sorted_indices[1]
        print("Secondo massimo: {0}".format(subset_df['Time'].iloc[x[second_max_index]]))

        maxima_info.append((id_sensor, subset_df['Time'].iloc[x[y == y.max()]], subset_df['Time'].iloc[x[second_max_index]]))

    return maxima_info

maxima_firstmodel_split2 = find_maxima(df_subset_2)

# Create dataframes for maxima information
#df_maxima_1 = pd.DataFrame(maxima_area_1, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])
maxima_firstmodel_split2 = pd.DataFrame(maxima_firstmodel_split2, columns=['Id_sensor', 'Primo_massimo', 'Secondo_massimo'])